In [1]:
import time
start_time = time.time()
import zipfile
import os
from pathlib import Path
from lightglue import LightGlue, SuperPoint
from lightglue.utils import load_image, rbd
import torch
import numpy as np



extract_dir = Path("images")
keypoints_dir = Path("Keypoints")
matches_dir = Path("Matches")
keypoints_dir.mkdir(exist_ok=True)
matches_dir.mkdir(exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
extractor = SuperPoint(max_num_keypoints=10000).eval().to(device)
matcher = LightGlue(features="superpoint").eval().to(device)
image_paths = sorted(extract_dir.glob("*.png")) 

for i in range(len(image_paths) - 1):
    image_path1 = image_paths[i]
    image_path2 = image_paths[i + 1]
    image1 = load_image(image_path1).to(device)
    image2 = load_image(image_path2).to(device)
    feats1 = extractor.extract(image1)
    feats2 = extractor.extract(image2)
    matches12 = matcher({"image0": feats1, "image1": feats2})
    feats1, feats2, matches12 = [rbd(x) for x in [feats1, feats2, matches12]]
    kpts1, kpts2, matches = feats1["keypoints"], feats2["keypoints"], matches12["matches"]
    keypoints_indices1 = np.arange(len(kpts1)).reshape(-1, 1)
    keypoints_indices2 = np.arange(len(kpts2)).reshape(-1, 1)
    np.savetxt(keypoints_dir / f"{image_path1.stem}.png.txt", kpts1.cpu().numpy(), fmt='%f')
    np.savetxt(keypoints_dir / f"{image_path2.stem}.png.txt", kpts2.cpu().numpy(), fmt='%f')
    matches_indices = np.hstack((matches[..., 0].reshape(-1, 1), matches[..., 1].reshape(-1, 1)))
    np.savetxt(matches_dir / f"{image_path1.stem}_{image_path2.stem}.txt", matches_indices, fmt='%d')
    
for i in range(1000000):
    pass

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time:.2f} seconds")


Elapsed time: 1671.11 seconds


In [2]:
import zipfile
import os
from pathlib import Path
from lightglue import LightGlue, SuperPoint
from lightglue.utils import load_image, rbd
import torch
import numpy as np

extract_dir = Path("images")
keypoints_dir = Path("Keypoints")
matches_dir = Path("Matches")
keypoints_dir.mkdir(exist_ok=True)
matches_dir.mkdir(exist_ok=True)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the extractor and matcher
extractor = SuperPoint(max_num_keypoints=10000).eval().to(device)
matcher = LightGlue(features="superpoint").eval().to(device)

# Get a sorted list of image paths
image_paths = sorted(extract_dir.glob("*.png"))  # Assuming images are in .jpg format

image_path1 = image_paths[419]
image_path2 = image_paths[0]

image1 = load_image(image_path1).to(device)
image2 = load_image(image_path2).to(device)

    # Extract features
feats1 = extractor.extract(image1)
feats2 = extractor.extract(image2)

    # Match features
matches12 = matcher({"image0": feats1, "image1": feats2})
feats1, feats2, matches12 = [rbd(x) for x in [feats1, feats2, matches12]]

    # Extract keypoints and matches
kpts1, kpts2, matches = feats1["keypoints"], feats2["keypoints"], matches12["matches"]

    # Save keypoints (indices)
keypoints_indices1 = np.arange(len(kpts1)).reshape(-1, 1)
keypoints_indices2 = np.arange(len(kpts2)).reshape(-1, 1)
np.savetxt(keypoints_dir / f"{image_path1.stem}_jpg.txt", kpts1.cpu().numpy(), fmt='%f')
np.savetxt(keypoints_dir / f"{image_path2.stem}_jpg.txt", kpts2.cpu().numpy(), fmt='%f')

   
matches_indices = np.hstack((matches[..., 0].reshape(-1, 1), matches[..., 1].reshape(-1, 1)))
np.savetxt(matches_dir / f"{image_path1.stem}_{image_path2.stem}.txt", matches_indices, fmt='%d')
